In [1]:
# load data from urls
import pandas as pd
population_url = 'https://www2.census.gov/programs-surveys/popest/tables/2010-2019/cities/totals/SUB-IP-EST2019-ANNRNK.xlsx'
rent_url = 'https://files.zillowstatic.com/research/public_v2/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv'
rent = pd.read_csv(rent_url)
population = pd.read_excel(population_url, skiprows=3, skipfooter=5)
# clean up the column names
population = population.rename(columns={'Unnamed: 0': 'Rank', 'Unnamed: 1': 'City'})
population = population.drop(columns=['Census', 'Estimates Base'])

In [2]:
population.head()

,Rank,City,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,"New York city, New York",8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,2,"Los Angeles city, California",3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576
2,3,"Chicago city, Illinois",2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976
3,4,"Houston city, Texas",2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268
4,5,"Phoenix city, Arizona",1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992


In [3]:
# function to parse the city column into city and state columns
def get_state_name(city):
  state = city.split(',')[1]
  state = state.strip()
  return state

def get_city_name(city):
  name = city.split(',')[0]
  name = name.split()
  result = name[0]
  for word in name[1:]:
    if word[0].isupper():
      result += ' '
      result += word
  return result

In [4]:
population['State'] = population['City'].apply(get_state_name)
population['City_Name'] = population['City'].apply(get_city_name)

In [5]:
# quick look at the rent df
rent = rent.drop(columns=['RegionID', 'SizeRank'])
rent = rent.drop(index=0)
print(rent.shape)
rent.head()

(101, 85)


,RegionName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
1,"New York, NY",2375.0,2387.0,2399.0,2411,2423,2434,2446,2458,2470,2481,2492,2503,2514,2523,2533,2543,2552,2560,2569,2577,2584,2592,2598,2604,2610,2614,2619,2624,2628,2633,2637,2641,2645,2649,2653,2657,2661,2665,2669,...,2686,2689,2692,2694,2697,2701,2704,2708,2712,2717,2721,2726,2731,2737,2742,2748,2754,2760,2766,2772,2778,2784,2790,2795,2800,2805,2803,2802,2801,2785,2769,2753,2734,2714,2694,2673,2651,2630,2607,2584.0
2,"Los Angeles-Long Beach-Anaheim, CA",1838.0,1855.0,1873.0,1890,1906,1923,1940,1956,1973,1989,2004,2018,2032,2044,2055,2067,2079,2091,2103,2115,2127,2139,2152,2164,2176,2188,2200,2211,2222,2233,2243,2253,2263,2274,2284,2294,2304,2313,2323,...,2376,2385,2393,2401,2409,2418,2426,2435,2443,2452,2461,2469,2478,2486,2494,2502,2510,2517,2525,2532,2539,2546,2553,2559,2565,2571,2573,2576,2578,2578,2577,2576,2574,2573,2572,2570,2568,2566,2564,2562.0
3,"Chicago, IL",1514.0,1520.0,1525.0,1531,1536,1542,1547,1553,1558,1563,1567,1572,1576,1580,1584,1588,1592,1597,1601,1606,1611,1616,1622,1627,1632,1637,1641,1645,1648,1652,1655,1658,1660,1663,1666,1668,1671,1674,1676,...,1691,1694,1696,1699,1701,1704,1707,1710,1713,1716,1719,1722,1725,1729,1732,1736,1740,1744,1748,1752,1756,1761,1765,1768,1772,1775,1776,1777,1777,1775,1772,1769,1766,1762,1758,1754,1750,1746,1742,1738.0
4,"Dallas-Fort Worth, TX",1209.0,1217.0,1224.0,1232,1239,1246,1254,1261,1268,1276,1282,1289,1296,1301,1307,1313,1318,1324,1329,1335,1340,1346,1351,1357,1363,1369,1375,1381,1387,1392,1398,1404,1409,1415,1419,1424,1429,1434,1438,...,1463,1467,1471,1475,1479,1482,1486,1490,1494,1498,1502,1506,1510,1515,1520,1524,1529,1534,1538,1543,1548,1552,1556,1560,1564,1567,1569,1571,1573,1574,1576,1577,1579,1581,1583,1585,1587,1589,1591,1593.0
5,"Philadelphia, PA",1340.0,1345.0,1349.0,1354,1358,1362,1366,1371,1375,1379,1383,1388,1392,1395,1398,1402,1405,1409,1412,1416,1420,1424,1427,1431,1434,1438,1441,1445,1448,1452,1455,1459,1462,1466,1469,1471,1474,1477,1479,...,1492,1494,1497,1500,1504,1507,1511,1515,1519,1524,1528,1532,1537,1541,1545,1550,1554,1559,1563,1567,1572,1576,1580,1584,1587,1591,1593,1596,1598,1600,1602,1604,1606,1608,1611,1613,1615,1617,1619,1621.0


In [6]:
# fix a few mis-named cities
rent.at[2, 'RegionName'] = 'Los Angeles, CA'
rent.at[4, 'RegionName'] = 'Dallas, TX'
rent.at[8, 'RegionName'] = 'Miami, FL'
rent.at[16, 'RegionName'] = 'Minneapolis, MN'
rent.at[42, 'RegionName'] = 'Louisville/Jefferson County, KY'

In [7]:
# parse city into city and state columns
rent['City_Name'] = rent['RegionName'].apply(get_city_name)
rent['State'] = rent['RegionName'].apply(get_state_name)

In [8]:
# the two dataframes
df = pd.merge(population, rent, on='City_Name')
df = df.drop(columns=['City', 'State_y'])
df = df.rename({'State_x': 'State'}, axis=1)

In [9]:
# rearange columns to look nicer
columns = list(df.columns)
columns.insert(1, columns[11])
columns.insert(1, columns[13])
columns.insert(3, columns[15])
columns.pop(14)
columns.pop(14)
columns.pop(14)

'RegionName'

In [10]:
df = df[columns]
# separate columns to prepare them for ml models
population_columns = columns[4:14]
rent_columns = columns[14:]
df = df.reset_index()
df.head()

,index,Rank,City_Name,State,RegionName,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
0,0,1,New York,New York,"New York, NY",8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817,2375.0,2387.0,2399.0,2411,2423,2434,2446,2458,2470,2481,2492,2503,2514,2523,2533,2543,2552,2560,2569,2577,2584,2592,2598,2604,2610,...,2686,2689,2692,2694,2697,2701,2704,2708,2712,2717,2721,2726,2731,2737,2742,2748,2754,2760,2766,2772,2778,2784,2790,2795,2800,2805,2803,2802,2801,2785,2769,2753,2734,2714,2694,2673,2651,2630,2607,2584.0
1,1,2,Los Angeles,California,"Los Angeles, CA",3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576,1838.0,1855.0,1873.0,1890,1906,1923,1940,1956,1973,1989,2004,2018,2032,2044,2055,2067,2079,2091,2103,2115,2127,2139,2152,2164,2176,...,2376,2385,2393,2401,2409,2418,2426,2435,2443,2452,2461,2469,2478,2486,2494,2502,2510,2517,2525,2532,2539,2546,2553,2559,2565,2571,2573,2576,2578,2578,2577,2576,2574,2573,2572,2570,2568,2566,2564,2562.0
2,2,3,Chicago,Illinois,"Chicago, IL",2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976,1514.0,1520.0,1525.0,1531,1536,1542,1547,1553,1558,1563,1567,1572,1576,1580,1584,1588,1592,1597,1601,1606,1611,1616,1622,1627,1632,...,1691,1694,1696,1699,1701,1704,1707,1710,1713,1716,1719,1722,1725,1729,1732,1736,1740,1744,1748,1752,1756,1761,1765,1768,1772,1775,1776,1777,1777,1775,1772,1769,1766,1762,1758,1754,1750,1746,1742,1738.0
3,3,4,Houston,Texas,"Houston, TX",2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268,1249.0,1261.0,1272.0,1284,1295,1307,1318,1329,1340,1351,1359,1366,1374,1378,1383,1387,1390,1393,1397,1399,1401,1404,1405,1407,1408,...,1424,1428,1432,1437,1442,1446,1450,1454,1456,1458,1460,1462,1463,1464,1466,1468,1469,1471,1473,1475,1478,1480,1482,1485,1487,1489,1489,1489,1489,1489,1489,1488,1488,1488,1488,1488,1488,1488,1488,1488.0
4,4,5,Phoenix,Arizona,"Phoenix, AZ",1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992,1008.0,1013.0,1017.0,1022,1027,1032,1037,1042,1047,1052,1058,1063,1069,1074,1080,1086,1091,1097,1102,1107,1113,1118,1124,1129,1135,...,1234,1239,1245,1252,1258,1265,1272,1280,1288,1297,1305,1313,1322,1331,1339,1348,1357,1365,1374,1383,1392,1401,1411,1419,1428,1436,1443,1450,1457,1464,1472,1480,1488,1497,1505,1514,1523,1531,1540,1549.0


In [11]:
# create funtion that will predict rent prices for the next three months
from fbprophet import Prophet
def project_rent(index):
    '''
    Creates a DataFrame (time_df) that is required by fbprohet model 
    to predict time series data.
    Returns the projected rent prices for the next three months (Jan, Feb, Mar 2021).
    '''
    # create df that fbprophet requires
    time_df = pd.DataFrame(df.loc[index, rent_columns]).reset_index()
    time_df['index'] = pd.to_datetime(time_df['index'], infer_datetime_format=True)
    time_df.columns = ['ds', 'y']
    # train the model
    projection = Prophet()
    projection.fit(time_df)
    future = projection.make_future_dataframe(periods=3, freq='MS')
    y_pred = projection.predict(future)
    return y_pred['trend'][-3:].values


In [12]:
# makes empty columns for rent projections
df['Jan Proj Rent'] = [0] * len(df)
df['Feb Proj Rent'] = [0] * len(df)
df['Mar Proj Rent'] = [0] * len(df)

In [ ]:
# train the model for all the cities
for city in df.index:
    df.loc[city, ['Jan Proj Rent', 'Feb Proj Rent', 'Mar Proj Rent']] = project_rent(city)

In [20]:
rent_columns[-3:]

['2020-10', '2020-11', '2020-12']

In [22]:
# slim down the dataframe into the needed columns for the API
result = df.drop(columns=['index', 'Rank'] + population_columns[:-1] + rent_columns[:-1])
result = result.rename({'City_Name': 'City', 2019: '2019-Population', '2020-12': 'Dec Avg Rent'}, axis=1)
print(result.shape)
result.head()

(114, 8)


,City,State,RegionName,2019-Population,Dec Avg Rent,Jan Proj Rent,Feb Proj Rent,Mar Proj Rent
0,New York,New York,"New York, NY",8336817,2584.0,2622.195108,2610.641187,2600.205387
1,Los Angeles,California,"Los Angeles, CA",3979576,2562.0,2572.626595,2572.932151,2573.208136
2,Chicago,Illinois,"Chicago, IL",2693976,1738.0,1747.096325,1745.352912,1743.778217
3,Houston,Texas,"Houston, TX",2320268,1488.0,1488.816284,1488.941250,1489.054122
4,Phoenix,Arizona,"Phoenix, AZ",1680992,1549.0,1553.823188,1561.939811,1569.270955


In [ ]:
# save the two different tables to a csv to be uploaded into the database
result[['City', 'State', 'RegionName', '2019-Population']].to_csv('cityspire_population.csv', index=False)
result[['City', 'State', 'RegionName', 'Dec Avg Rent', 'Jan Proj Rent', 'Feb Proj Rent', 'Mar Proj Rent']].to_csv('cityspire_rent.csv', index=False)